---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [11]:
# import data
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
train_df = pd.read_csv('readonly/train.csv',encoding = 'ISO-8859-1')
train_df = pd.DataFrame(train_df)
test_df =  pd.read_csv('readonly/test.csv',encoding = 'ISO-8859-1')
test_df = pd.DataFrame(test_df)
ticket_id = test_df['ticket_id']


# first preprocessing function
def my_preprocessing_clean(a):
    
    # Leakage or strings
    del (a['grafitti_status'],
    a['violation_zip_code'],a['violation_description'],
    a['violation_street_number'],a['mailing_address_str_number'],
    a['ticket_id'],a['violator_name'],a['inspector_name'],
    a['non_us_str_code'],a['city'],
    a['state'],a['country'])
    
    # Fill Nans
    a = a.fillna(method = 'ffill')
    
    return a
    
# Recode, Diffs between issued date and hearing date 
def YMD(a,b): # a function to recode dates
    a = a.str.split()
    b = b.str.split()
    YMD_a = []
    YMD_b = []
    for c in a:
        YMD_a.append(c[0])
    for d in b:
        YMD_b.append(d[0])
    from datetime import datetime
    def days_between(d1, d2):
        d1 = datetime.strptime(d1, "%Y-%m-%d")
        d2 = datetime.strptime(d2, "%Y-%m-%d")
        return abs((d2 - d1).days)
    Diffs = []
    for i in np.arange(len(YMD_a)):
        Diffs.append(days_between(YMD_a[i],YMD_b[i]))
    return Diffs
    

# Second preprocessing function
def my_preprocessing_recode(a):
    # Remove "nonsig" features according to initial random forest
    if np.array(a.columns)[-1] != 'compliance':
        relevant_feature = ['violation_street_name', 'mailing_address_str_name','zip_code','diffs_ticket_hear','disposition', 
                        'late_fee']
    else:relevant_feature = ['violation_street_name', 'mailing_address_str_name','zip_code','diffs_ticket_hear','disposition', 
                        'late_fee','compliance']
    a = a[relevant_feature]
    
    # Recode strings to integers
    from sklearn.preprocessing import LabelEncoder
    le_disposition = LabelEncoder().fit(a['disposition'])
    le_str_name = LabelEncoder().fit(a['violation_street_name'])
    le_mailing_name = LabelEncoder().fit(a['mailing_address_str_name'])
    
    for le,attr in (zip([le_disposition,le_str_name,le_mailing_name],
                    ['disposition','violation_street_name','mailing_address_str_name'])):
        a[attr] = le.transform(a[attr])
        
    return a

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Deploy Function 1
train_df_nan= train_df
train_df_nan = my_preprocessing_clean(train_df_nan)

# re-arrange columns
train_df_nan['ticket_issued_date'] = YMD(train_df_nan['ticket_issued_date'],train_df_nan['hearing_date'])
train_df_nan = train_df_nan.rename(columns={'ticket_issued_date':'diffs_ticket_hear'})
del train_df_nan['hearing_date']
    
# Deploy Function 2    
train_df_nan = my_preprocessing_recode(train_df_nan)

# Remove zipcodes that are not readable 
train_df_nan = train_df_nan[train_df_nan['zip_code'].apply(lambda x: str(x).isdigit())] # ATTENTION! 
# Try to use list comprehension


# ----/// Below: machine learning process ///---- 
# split data

X_train_df = train_df_nan.iloc[:,:-1]
y_train_df = train_df_nan.iloc[:,-1]

X_train,X_test,y_train,y_test = train_test_split(X_train_df,y_train_df,random_state = 0)

# train the date in the training set

clf = RandomForestClassifier(n_estimators= 80,random_state=0).fit(X_train,y_train)

from sklearn.metrics import roc_curve,auc

y_pred_prob = clf.predict_proba(X_test)[:,1]
fpr_rf,tpr_rf,_ = roc_curve(y_test,y_pred_prob)
roc_auc_rf = auc(fpr_rf,tpr_rf)
print('AUC returns value of {:.2f}'.format(roc_auc_rf))
# Have checked feature importance and stored >=.05 in the variable "relevant_feature"

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AUC returns value of 0.80


In [12]:
# Preprocessing of test data
test_df_nan = test_df
test_df_nan = my_preprocessing_clean(test_df_nan)
test_df_nan['ticket_issued_date'] = YMD(test_df_nan['ticket_issued_date'],test_df_nan['hearing_date'])
test_df_nan = test_df_nan.rename(columns={'ticket_issued_date':'diffs_ticket_hear'})
test_df_nan = my_preprocessing_recode(test_df_nan)

# Do not know why. But it does not work on the training set
from sklearn.preprocessing import LabelEncoder

le_zipcode = LabelEncoder().fit(test_df_nan['zip_code'])
test_df_nan['zip_code'] = le_zipcode.transform(test_df_nan['zip_code'])

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
y_test_pred_prob = pd.Series(clf.predict_proba(test_df_nan)[:,1])
y_test_pred_prob.set_axis(labels = ticket_id,axis = 0)
y_test_pred_prob.rename('compliance',inplace=True)

ticket_id
284932    0.275000
285362    0.256250
285361    0.337500
285338    0.312500
285346    0.300000
285345    0.325000
285347    0.262500
285342    0.887500
285530    0.206250
284989    0.275000
285344    0.337500
285343    0.250000
285340    0.218750
285341    0.237500
285349    0.225000
285348    0.275000
284991    0.187500
285532    0.168750
285406    0.187500
285001    0.231250
285006    0.193750
285405    0.187500
285337    0.231250
285496    0.325000
285497    0.425000
285378    0.193750
285589    0.162500
285585    0.262500
285501    0.375000
285581    0.150000
            ...   
376367    0.256250
376366    0.306250
376362    0.087500
376363    0.062500
376365    0.256250
376364    0.306250
376228    0.237500
376265    0.231250
376286    0.712500
376320    0.312500
376314    0.275000
376327    0.787500
376385    0.812500
376435    0.850000
376370    0.850000
376434    0.137500
376459    0.450000
376478    0.225000
376473    0.320833
376484    0.306250
376482    0.275000
37

ticket_id
284932    0.275000
285362    0.256250
285361    0.337500
285338    0.312500
285346    0.300000
285345    0.325000
285347    0.262500
285342    0.887500
285530    0.206250
284989    0.275000
285344    0.337500
285343    0.250000
285340    0.218750
285341    0.237500
285349    0.225000
285348    0.275000
284991    0.187500
285532    0.168750
285406    0.187500
285001    0.231250
285006    0.193750
285405    0.187500
285337    0.231250
285496    0.325000
285497    0.425000
285378    0.193750
285589    0.162500
285585    0.262500
285501    0.375000
285581    0.150000
            ...   
376367    0.256250
376366    0.306250
376362    0.087500
376363    0.062500
376365    0.256250
376364    0.306250
376228    0.237500
376265    0.231250
376286    0.712500
376320    0.312500
376314    0.275000
376327    0.787500
376385    0.812500
376435    0.850000
376370    0.850000
376434    0.137500
376459    0.450000
376478    0.225000
376473    0.320833
376484    0.306250
376482    0.275000
37

In [43]:
def blight_model():
    
    # Your code here
    
    return y_test_pred_prob

In [44]:
blight_model()

ticket_id
284932    0.275000
285362    0.256250
285361    0.337500
285338    0.312500
285346    0.300000
285345    0.325000
285347    0.262500
285342    0.887500
285530    0.206250
284989    0.275000
285344    0.337500
285343    0.250000
285340    0.218750
285341    0.237500
285349    0.225000
285348    0.275000
284991    0.187500
285532    0.168750
285406    0.187500
285001    0.231250
285006    0.193750
285405    0.187500
285337    0.231250
285496    0.325000
285497    0.425000
285378    0.193750
285589    0.162500
285585    0.262500
285501    0.375000
285581    0.150000
            ...   
376367    0.256250
376366    0.306250
376362    0.087500
376363    0.062500
376365    0.256250
376364    0.306250
376228    0.237500
376265    0.231250
376286    0.712500
376320    0.312500
376314    0.275000
376327    0.787500
376385    0.812500
376435    0.850000
376370    0.850000
376434    0.137500
376459    0.450000
376478    0.225000
376473    0.320833
376484    0.306250
376482    0.275000
37